# GPX Export for Outdooractive

This notebook demonstrates how to:
1. Select a rectangular area on a map of Norway
2. Filter trails that intersect with the selected area
3. Export the trails to GPX format suitable for Outdooractive import

## 1. Setup and Data Loading

In [ ]:
import sys
from pathlib import Path

import geopandas as gpd

# Add project root to path
project_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import our modules
from trails.io.export.area_selector import (  # noqa: E402
    create_interactive_selection_map,
    save_selection_preset,
)
from trails.io.export.gpx import filter_trails_by_bbox  # noqa: E402
from trails.io.sources.geonorge import Language, Source  # noqa: E402

print("✓ Modules loaded")

In [ ]:
# Load trail data (using cache in repo root)
print("Loading trail data...")
source = Source(cache_dir=str(project_root / ".cache"))
trail_data = source.load_turrutebasen(language=Language.EN)

# Get the main trail layer
trails = trail_data.spatial_layers["hiking_trail_centerline"]
print(f"\nLoaded {len(trails):,} trail segments")
print(f"CRS: {trails.crs}")
print(f"Total bounds: {trails.total_bounds}")

## 2. Interactive Area Selection

Use the map below to select your area of interest:

## 2. Interactive Area Selection

### ✨ Automatic Bounds Capture

1. **Search for a city** using the magnifying glass icon
2. **Draw a rectangle** around your area of interest
3. **Bounds are captured automatically** - no copying needed!

The selected bounds will be available in the next cell.

In [ ]:
# Create interactive selection map with automatic bounds capture
map_widget, bounds_container = create_interactive_selection_map(trails, zoom_start=5)

In [ ]:
# Load a saved preset and display it on the map (optional)
from trails.io.export.area_selector import load_selection_preset

# Try to load an existing preset
preset_name = "gjerstad_nissedal"
preset_file = project_root / ".cache" / f"area_presets_{preset_name}.json"

if preset_file.exists():
    loaded_bounds = load_selection_preset(preset_name, presets_file=preset_file)
    if loaded_bounds:
        # Use the map's built-in function to display the preset
        bounds_container["load_preset"](loaded_bounds)
        print(f"✅ Loaded and displayed preset: {preset_name}")
        print(f"   Bounds: {loaded_bounds}")
    else:
        print(f"ℹ️ No preset found with name: {preset_name}")
else:
    print("ℹ️ No presets file found. Draw an area and save it first.")

In [ ]:
# Display the interactive map
map_widget

### Load and Display Saved Presets

You can load previously saved area presets and visualize them on the map:

In [ ]:
# Save the current selection as a preset
selected_bounds = bounds_container["bounds"]

if selected_bounds:
    save_selection_preset(preset_name, selected_bounds, presets_file=preset_file)
    print("✓ Area preset saved!")
    print(f"  Name: {preset_name}")
    print(f"  Bounds: {selected_bounds}")
    print(f"  File: {preset_file}")
else:
    print("⚠️ No area selected yet! Please draw a rectangle on the map above.")

## 4. Process Selected Area

In [ ]:
# Get the automatically captured bounds
selected_bounds = bounds_container["bounds"]

if selected_bounds:
    print(f"✅ Using auto-captured bounds: {selected_bounds}")
    print(f"  Longitude: {selected_bounds[0]:.2f} to {selected_bounds[2]:.2f}")
    print(f"  Latitude: {selected_bounds[1]:.2f} to {selected_bounds[3]:.2f}")
else:
    print("⚠️ No area selected yet! Please draw a rectangle on the map above.")
    print("\nAlternatively, you can manually set bounds:")
    print("# selected_bounds = (8.5, 58.8, 9.2, 59.2)  # Gjerstad/Nissedal")
    print("# selected_bounds = (10.3, 59.8, 11.0, 60.2)  # Oslo")
    print("# selected_bounds = (5.0, 60.2, 5.8, 60.6)  # Bergen")

    # Default to Gjerstad/Nissedal for demo
    selected_bounds = (8.5, 58.8, 9.2, 59.2)
    print("\n📍 Using default area: Gjerstad/Nissedal")

In [ ]:
# Filter trails by selected area
if selected_bounds:
    # Convert bounds to trail CRS
    from shapely.geometry import box

    bounds_gdf = gpd.GeoDataFrame([1], geometry=[box(*selected_bounds)], crs="EPSG:4326")

    if trails.crs != "EPSG:4326":
        bounds_gdf = bounds_gdf.to_crs(trails.crs)  # type: ignore[arg-type]
        area_bounds = bounds_gdf.geometry[0].bounds
    else:
        area_bounds = selected_bounds

    # Filter with optional buffer
    buffer_km = 5  # Add 5km buffer around selection
    filtered_trails = filter_trails_by_bbox(trails, area_bounds, buffer_m=buffer_km * 1000)

    print("\n📊 Filtering results:")
    print(f"  Original trails: {len(trails):,}")
    print(f"  Filtered trails: {len(filtered_trails):,}")
    print(f"  Buffer applied: {buffer_km} km")

    # Calculate statistics
    total_length_km = filtered_trails.geometry.length.sum() / 1000
    print(f"  Total length: {total_length_km:,.1f} km")

## 4. Enrich Trail Data

Add names and metadata from attribute tables:

In [ ]:
# Join with attribute data
hiking_attrs = trail_data.attribute_tables["hiking_trail_info_table"]

# Join filtered trails with attributes
enriched_trails = filtered_trails.merge(
    hiking_attrs[
        [
            "hiking_trail_fk",
            "trail_number",
            "trail_name",
            "special_hiking_trail_type",
            "maintenance_responsible",
            "difficulty",
        ]
    ],
    left_on="local_id",
    right_on="hiking_trail_fk",
    how="left",
)

print("\n📝 Trail data enrichment:")
print(f"  Trails with numbers: {enriched_trails['trail_number'].notna().sum():,}")
print(f"  Trails with names: {enriched_trails['trail_name'].notna().sum():,}")
print(f"  Trails with types: {enriched_trails['special_hiking_trail_type'].notna().sum():,}")
print(f"  Trails with maintainer: {enriched_trails['maintenance_responsible'].notna().sum():,}")
print(f"  Trails with difficulty: {enriched_trails['difficulty'].notna().sum():,}")

# Show sample
print("\n Sample trails:")
sample = enriched_trails[enriched_trails["trail_name"].notna()].head(5)
for _, trail in sample.iterrows():
    print(f"  • {trail['trail_name']}: {trail.geometry.length / 1000:.1f} km")

In [ ]:
import pandas as pd

# Look for a specific trail by examining trail_number patterns
print("🔍 Examining specific trail segments")
print("=" * 60)

# Find trails with the same trail_number to understand connectivity issues
trail_counts = enriched_trails["trail_number"].value_counts()
print("Most common trail numbers (these have multiple segments):")
for trail_num, count in trail_counts.head(10).items():
    # Cast to str for type checking
    trail_num_str = str(trail_num) if trail_num is not None else None
    if trail_num_str and trail_num_str != "nan":
        # Get the trail name for this number
        trail_name = enriched_trails[enriched_trails["trail_number"] == trail_num]["trail_name"].iloc[0]
        print(f"  {trail_num}: {count} segments - {trail_name if pd.notna(trail_name) else 'No name'}")

# Pick one trail to analyze in detail
if len(trail_counts) > 0:
    # Get the trail with most segments
    example_trail_num = trail_counts.index[0]
    example_segments = enriched_trails[enriched_trails["trail_number"] == example_trail_num].copy()

    print(f"\n📍 Detailed analysis of trail '{example_trail_num}':")
    print(f"  Name: {example_segments['trail_name'].iloc[0]}")
    print(f"  Segments: {len(example_segments)}")

    # Check connectivity between these segments
    from trails.io.export import find_connected_segments

    # Reset index for connectivity analysis
    example_segments_reset = example_segments.reset_index(drop=True)

    print("\n  Connectivity test:")
    for tolerance in [5, 10, 25, 50]:
        components = find_connected_segments(example_segments_reset, tolerance_m=tolerance)
        print(f"    {tolerance}m tolerance: {len(components)} component(s)")
        if len(components) > 1:
            print(f"      Sizes: {[len(c) for c in components]}")

    # Calculate distances between segment endpoints
    print("\n  Endpoint distances (sample):")
    from shapely.geometry import Point

    # Get endpoints
    endpoints = []
    for idx, row in example_segments_reset.iterrows():
        geom = row.geometry
        if geom.geom_type == "MultiLineString":
            first_line = geom.geoms[0]
            last_line = geom.geoms[-1]
            start = Point(first_line.coords[0])
            end = Point(last_line.coords[-1])
        else:
            start = Point(geom.coords[0])
            end = Point(geom.coords[-1])
        endpoints.append((idx, start, end))

    # Calculate minimum distances between different segments
    min_distances = []
    for i in range(len(endpoints)):
        for j in range(i + 1, len(endpoints)):
            dist1 = endpoints[i][1].distance(endpoints[j][1]) * 111000  # Rough conversion to meters
            dist2 = endpoints[i][1].distance(endpoints[j][2]) * 111000
            dist3 = endpoints[i][2].distance(endpoints[j][1]) * 111000
            dist4 = endpoints[i][2].distance(endpoints[j][2]) * 111000
            min_dist = min(dist1, dist2, dist3, dist4)
            min_distances.append((i, j, min_dist))

    # Show closest pairs
    min_distances.sort(key=lambda x: x[2])
    print("    Closest segment pairs:")
    for i, j, dist in min_distances[:5]:
        print(f"      Segments {i} and {j}: {dist:.1f} meters apart")

## 5. Choose Export Mode for Outdooractive

Select the export format that works best for your needs:

In [ ]:
# Choose export mode
print("🎯 Export Mode Selection for Outdooractive\n")
print("=" * 60)
print("\n1️⃣  SINGLE TRACK MODE")
print("   • All trails combined in ONE track with multiple segments")
print("   • Imports as a single entity to Outdooractive")
print("   • ⚠️ May show waypoints at segment boundaries")
print("   • File: trails_single_track.gpx")

print("\n2️⃣  MULTI-TRACK MODE (Original format)")
print("   • Each trail as a separate track")
print("   • May not display well in Outdooractive My Map")
print("   • File: trails_multi_track.gpx")

print("\n3️⃣  ZIP ARCHIVE MODE (Basic)")
print("   • Each trail segment as separate GPX file")
print("   • Simple export, no grouping")
print("   • File: trails_archive.zip")

print("\n4️⃣  SMART ZIP MODE ✨ (Recommended)")
print("   • Groups segments by trail_number")
print("   • Connected segments → single GPX file")
print("   • Disconnected segments → separate parts")
print("   • Best for Outdooractive Mass Import!")
print("   • File: trails_smart.zip")

print("\n" + "=" * 60)

# Set your choice here (1, 2, 3, or 4)
EXPORT_MODE = 4  # Default to smart ZIP mode

mode_names = {1: "Single Track", 2: "Multi-Track", 3: "Basic ZIP", 4: "Smart ZIP"}
print(f"\n✅ Selected: {mode_names[EXPORT_MODE]} Mode")

## 6. Export to GPX

Generate GPX file optimized for Outdooractive:

In [ ]:
# Import the export functions
from trails.io.export import (  # noqa: E402
    export_to_gpx,
    export_to_gpx_single_track,
    export_to_gpx_zip,
    export_to_gpx_zip_smart,
)

# Export based on selected mode
output_dir = project_root / "output"
output_dir.mkdir(exist_ok=True)

# Common export parameters
desc_fields = [
    "trail_number",
    "special_hiking_trail_type",
    "difficulty",
    "marking",
    "season",
    "trail_follows",
    "maintenance_responsible",
    "accuracy",
    "data_capture_date",
    "update_date",
]

print(f"🔧 Export Mode: {mode_names[EXPORT_MODE]}")
print("-" * 40)

if EXPORT_MODE == 1:
    # Single track mode
    gpx_file = output_dir / "trails_single_track.gpx"
    print("📦 Exporting as single track with multiple segments...")

    output_path, stats = export_to_gpx_single_track(
        enriched_trails,
        gpx_file,
        track_name=f"Trail Network - {preset_name.replace('_', ' ').title() if 'preset_name' in locals() else 'Selected Area'}",
        name_field="trail_name",
        desc_fields=desc_fields,
        simplify_tolerance=0.00001,
        max_trails=None,
    )

    print(f"\n✅ Export complete: {gpx_file.name}")
    print(f"   • Single track containing {stats.get('total_segments', 0)} segments")
    print(f"   • Total points: {stats.get('total_points', 0):,}")

elif EXPORT_MODE == 2:
    # Multi-track mode
    gpx_file = output_dir / "trails_multi_track.gpx"
    print("📦 Exporting as multiple tracks...")

    output_path, stats = export_to_gpx(
        enriched_trails,
        gpx_file,
        name_field="trail_name",
        desc_fields=desc_fields,
        simplify_tolerance=0.00001,
        max_trails=None,
    )

    print(f"\n✅ Export complete: {gpx_file.name}")
    print(f"   • Tracks exported: {stats['total_trails']}")
    print(f"   • Total points: {stats.get('total_points', 0):,}")

elif EXPORT_MODE == 3:
    # Basic ZIP archive mode
    gpx_file = output_dir / "trails_archive.zip"
    print("📦 Exporting as basic ZIP archive...")

    output_path, stats = export_to_gpx_zip(
        enriched_trails,
        gpx_file,
        name_field="trail_name",
        desc_fields=desc_fields,
        simplify_tolerance=0.00001,
        max_trails=None,
        group_by="special_hiking_trail_type",
    )

    print(f"\n✅ Export complete: {gpx_file.name}")
    print(f"   • Files in archive: {stats['total_files']}")
    print(f"   • Groups/folders: {len(stats.get('groups', []))}")

elif EXPORT_MODE == 4:
    # Smart ZIP mode with trail grouping
    gpx_file = output_dir / "trails_smart.zip"
    print("📦 Exporting as Smart ZIP with trail grouping...")
    print("   Grouping segments by trail number...")

    # Check how many trails have numbers
    trails_with_numbers = enriched_trails["trail_number"].notna().sum()
    print(f"   • Trails with numbers: {trails_with_numbers}/{len(enriched_trails)}")

    output_path, stats = export_to_gpx_zip_smart(
        enriched_trails,
        gpx_file,
        group_field="trail_number",  # Group by trail number
        name_field="trail_name",
        desc_fields=desc_fields,
        simplify_tolerance=0.00001,
        max_trails=None,
        folder_by=None,  # Don't use folders for cleaner import
    )

    print(f"\n✅ Export complete: {gpx_file.name}")
    print(f"   • Input segments: {stats['total_segments']}")
    print(f"   • Output GPX files: {stats['total_files']}")
    print(f"   • Merged trails: {stats['merged_trails']} (connected segments combined)")
    print(f"   • Split trails: {stats['split_trails']} (disconnected parts separated)")
    print(f"   • Skipped segments: {stats.get('skipped_segments', 0)}")

    # Show compression ratio
    if stats["total_segments"] > 0:
        ratio = stats["total_files"] / stats["total_segments"]
        print(f"   • Compression: {stats['total_segments']} → {stats['total_files']} ({ratio:.1%})")

# Common statistics
print(f"\n📊 File size: {stats['file_size_mb']:.2f} MB")

# Import instructions
print("\n" + "=" * 60)
print("📱 Import Instructions for Outdooractive:\n")

if EXPORT_MODE == 1:
    print("For SINGLE TRACK import:")
    print("1. Go to Outdooractive Route Planner")
    print("2. Click 'Import GPX'")
    print("3. Upload trails_single_track.gpx")
    print("Note: May show waypoint markers at segment boundaries")

elif EXPORT_MODE == 2:
    print("For MULTI-TRACK import:")
    print("1. Go to Outdooractive Route Planner")
    print("2. Click 'Import GPX'")
    print("3. Upload trails_multi_track.gpx")
    print("Note: Each trail appears separately")

elif EXPORT_MODE in [3, 4]:
    print("For ZIP ARCHIVE import:")
    print("1. Log into Outdooractive.com")
    print("2. Go to your profile → 'GPX Import' or 'Import'")
    print("3. Use the 'Mass GPX Importer' feature")
    print(f"4. Upload {gpx_file.name}")
    print("5. All trails will appear in 'My Routes'")
    print("\nBenefits of Smart ZIP:")
    print("• Each GPX represents a complete trail")
    print("• Connected segments are pre-merged")
    print("• Clean import without duplicate segments")

print("\n" + "=" * 60)

## 7. Verify GPX Export

Quick validation and visualization of the generated file:

In [ ]:
# Verify and visualize the export with interactive selection
import zipfile

import folium
from IPython.display import display
from lxml import etree

print("📋 Export Verification:")
print("=" * 50)

# Handle both ZIP and GPX modes
if EXPORT_MODE in [3, 4]:  # ZIP modes
    # Handle ZIP file verification
    if gpx_file.exists() and gpx_file.suffix == ".zip":
        print(f"✅ ZIP file created: {gpx_file.name}")
        print(f"   Size: {gpx_file.stat().st_size / 1024:.1f} KB")

        # Analyze ZIP contents
        with zipfile.ZipFile(gpx_file, "r") as zf:
            files = zf.namelist()
            gpx_files = [f for f in files if f.endswith(".gpx")]

            print("\n📦 ZIP Contents:")
            print(f"   Total files: {len(files)}")
            print(f"   GPX files: {len(gpx_files)}")

            # Show ALL GPX files
            print("\n   All GPX files in archive:")
            for i, gpx in enumerate(sorted(gpx_files), 1):
                size_kb = zf.getinfo(gpx).file_size / 1024
                print(f"     {i:3}. {gpx:<50} ({size_kb:6.1f} KB)")

            # Store trail data for visualization
            viz_trail_data: dict[str, dict] = {}

            # Parse all GPX files and store their data
            print("\n   Loading trail data for visualization...")
            for gpx_filename in sorted(gpx_files):
                try:
                    # Read GPX from ZIP
                    gpx_content = zf.read(gpx_filename)

                    # Parse GPX
                    root = etree.fromstring(gpx_content)
                    ns = {"gpx": "http://www.topografix.com/GPX/1/1"}

                    # Extract all tracks with their names and descriptions
                    tracks = root.xpath("//gpx:trk", namespaces=ns)
                    track_details = []

                    for track in tracks:
                        track_name_elem = track.find("gpx:name", ns)
                        track_desc_elem = track.find("gpx:desc", ns)
                        track_name = track_name_elem.text if track_name_elem is not None else "Unknown"
                        track_desc = track_desc_elem.text if track_desc_elem is not None else ""

                        # Get track segments for this track
                        track_segments = track.xpath(".//gpx:trkseg", namespaces=ns)
                        for segment in track_segments:
                            points = segment.xpath(".//gpx:trkpt", namespaces=ns)
                            if points:
                                coords = []
                                for point in points:
                                    lat = float(point.get("lat"))
                                    lon = float(point.get("lon"))
                                    coords.append([lat, lon])
                                if coords:
                                    track_details.append({"name": track_name, "desc": track_desc, "coords": coords})

                    # Extract trail number from filename
                    trail_number = None
                    if "(" in gpx_filename and ")" in gpx_filename:
                        trail_number = gpx_filename[gpx_filename.rindex("(") + 1 : gpx_filename.rindex(")")]

                    # Get the main track name (first one or from filename)
                    main_name = tracks[0].find("gpx:name", ns).text if tracks else gpx_filename

                    if track_details:
                        size_kb = zf.getinfo(gpx_filename).file_size / 1024
                        viz_trail_data[gpx_filename] = {
                            "name": main_name,
                            "trail_number": trail_number,
                            "track_details": track_details,
                            "segments": [td["coords"] for td in track_details],
                            "size_kb": size_kb,
                        }

                except Exception as e:
                    print(f"     Warning: Could not process {gpx_filename}: {e}")

            print(f"   ✅ Loaded {len(viz_trail_data)} trails successfully")

            # Create map with all trails as separate layers
            print("\n🗺️ Creating interactive map with trail selection...")
            print("   ➡️ Click the layer control icon (top-right) to show/hide trails")
            print("   ➡️ Click on trail segments to see full metadata")
            print("   ➡️ All trails are selected by default")
            print("-" * 60)

            # Create map centered on selected area
            if selected_bounds:
                center_lat = (selected_bounds[1] + selected_bounds[3]) / 2
                center_lon = (selected_bounds[0] + selected_bounds[2]) / 2
            else:
                center_lat, center_lon = 59.0, 10.0

            # Create map with topographic basemap
            gpx_map = folium.Map(
                location=[center_lat, center_lon],
                zoom_start=10,
                tiles=None,  # We'll add custom tiles
                prefer_canvas=True,  # Better performance with many layers
            )

            # Add topographic tile layer (OpenTopoMap)
            folium.TileLayer(
                tiles="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
                attr=(
                    'Map data: &copy; <a href="https://openstreetmap.org/copyright">OpenStreetMap</a> contributors, '
                    '<a href="http://viewfinderpanoramas.org">SRTM</a> | Map style: &copy; '
                    '<a href="https://opentopomap.org">OpenTopoMap</a> '
                    '(<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)'
                ),
                name="Topographic",
                overlay=False,
                control=True,
                max_zoom=17,
            ).add_to(gpx_map)

            # Add selected bounds rectangle
            if selected_bounds:
                bounds_rect = folium.Rectangle(
                    bounds=[
                        [selected_bounds[1], selected_bounds[0]],
                        [selected_bounds[3], selected_bounds[2]],
                    ],
                    color="#0066FF",  # Bright blue
                    weight=3,
                    fill=True,
                    fillColor="#0066FF",
                    fillOpacity=0.1,
                    popup="Selected export area",
                    name="📍 Export Area",
                )
                bounds_rect.add_to(gpx_map)

            # High contrast colors that work well on topographic maps
            # These colors are specifically chosen for visibility on terrain/topo backgrounds
            colors = [
                "#FF0000",  # Bright red
                "#00FF00",  # Bright green
                "#FF00FF",  # Magenta
                "#00FFFF",  # Cyan
                "#FFFF00",  # Yellow
                "#FF8800",  # Orange
                "#0088FF",  # Sky blue
                "#FF0088",  # Hot pink
                "#88FF00",  # Lime
                "#8800FF",  # Purple
                "#FF4444",  # Light red
                "#44FF44",  # Light green
                "#4444FF",  # Light blue
                "#FFAA00",  # Gold
                "#FF00AA",  # Deep pink
                "#00AAFF",  # Light cyan
                "#AA00FF",  # Violet
                "#00FFAA",  # Aquamarine
                "#AAFF00",  # Yellow-green
                "#FF5500",  # Dark orange
            ]

            # Add each trail as a separate feature group
            print("\n   Adding trails to map:")
            for i, (filename, data) in enumerate(sorted(viz_trail_data.items()), 1):
                trail_color = colors[i % len(colors)]

                # Create display name with trail number
                if data["trail_number"]:
                    display_name = f"{data['trail_number']}: {data['name'][:25]}"
                else:
                    display_name = f"{i}. {data['name'][:30]}"

                if len(data["name"]) > 30:
                    display_name += "..."

                # Create a feature group for this trail - ALL trails shown by default
                trail_group = folium.FeatureGroup(
                    name=display_name,
                    show=True,  # ALL trails shown by default
                    overlay=True,
                    control=True,
                )

                # Add all segments of this trail with thicker lines for better visibility
                if "track_details" in data:
                    # New format with track details
                    for _seg_idx, track_detail in enumerate(data["track_details"], 1):
                        # Parse description to extract metadata
                        desc_parts = track_detail["desc"].split(" | ") if track_detail["desc"] else []

                        # Create detailed popup HTML
                        popup_html = f"""<div style='font-family: sans-serif; width: 350px;'>
                        <h4 style='margin: 0 0 10px 0; color: #333;'>{track_detail["name"]}</h4>
                        <hr style='margin: 5px 0;'>
                        <b>File:</b> {filename}<br>
                        <b>Size:</b> {data["size_kb"]:.1f} KB<br>
                        <hr style='margin: 5px 0;'>
                        <b>Segment Details:</b><br>
                        <table style='width: 100%; font-size: 12px;'>"""

                        # Add each description part as a table row
                        for part in desc_parts:
                            if ":" in part:
                                key, value = part.split(":", 1)
                                popup_html += (
                                    f"<tr><td style='padding: 2px 5px 2px 0; vertical-align: top;'>"
                                    f"<b>{key.strip()}:</b></td>"
                                    f"<td style='padding: 2px;'>{value.strip()}</td></tr>"
                                )
                            else:
                                popup_html += f"<tr><td colspan='2' style='padding: 2px;'>{part.strip()}</td></tr>"

                        popup_html += "</table></div>"

                        # Add a white background line for contrast
                        folium.PolyLine(
                            locations=track_detail["coords"],
                            color="white",
                            weight=5,
                            opacity=0.8,
                        ).add_to(trail_group)

                        # Add the colored trail line on top
                        folium.PolyLine(
                            locations=track_detail["coords"],
                            color=trail_color,
                            weight=3,
                            opacity=1.0,
                            popup=folium.Popup(popup_html, max_width=400),
                        ).add_to(trail_group)
                else:
                    # Fallback for old format
                    for segment_coords in data["segments"]:
                        folium.PolyLine(
                            locations=segment_coords,
                            color="white",
                            weight=5,
                            opacity=0.8,
                        ).add_to(trail_group)

                        folium.PolyLine(
                            locations=segment_coords,
                            color=trail_color,
                            weight=3,
                            opacity=1.0,
                            popup=folium.Popup(
                                f"<b>{data['name']}</b><br>"
                                f"Trail: {data['trail_number'] or 'N/A'}<br>"
                                f"File: {filename}<br>"
                                f"Size: {data['size_kb']:.1f} KB<br>"
                                f"Segments: {len(data['segments'])}",
                                max_width=300,
                            ),
                        ).add_to(trail_group)

                trail_group.add_to(gpx_map)

                if i <= 5:
                    print(f"     ✓ {display_name} ({data['size_kb']:.1f} KB) - {trail_color}")

            if len(viz_trail_data) > 5:
                print(f"     ... and {len(viz_trail_data) - 5} more trails")

            print(f"   ✅ All {len(viz_trail_data)} trails added and selected")

            # Add layer control - collapsed by default, user clicks to expand
            folium.LayerControl(
                collapsed=True,  # Start collapsed - click to expand
                autoZIndex=True,
                position="topright",
            ).add_to(gpx_map)

            # Display the map without the info box
            print("\n🗺️ Interactive Map (Topographic):")
            display(gpx_map)

            # Show manifest if present
            if "MANIFEST.txt" in files:
                print("\n📄 Manifest Summary:")
                manifest = zf.read("MANIFEST.txt").decode("utf-8")
                summary_lines = manifest.split("\n")[-10:]
                for line in summary_lines:
                    if line.strip():
                        print(f"   {line}")
    else:
        print("❌ ZIP file not found")

else:  # GPX modes (1 or 2)
    # Handle single GPX file verification
    if gpx_file.exists() and gpx_file.suffix == ".gpx":
        print(f"✅ GPX file created: {gpx_file.name}")
        print(f"   Size: {gpx_file.stat().st_size / (1024 * 1024):.2f} MB")

        # Parse GPX
        with open(gpx_file, "rb") as f:
            tree = etree.parse(f)
            root = tree.getroot()

        # Count elements
        ns = {"gpx": "http://www.topografix.com/GPX/1/1"}
        tracks = root.xpath("//gpx:trk", namespaces=ns)
        segments = root.xpath("//gpx:trkseg", namespaces=ns)
        points = root.xpath("//gpx:trkpt", namespaces=ns)

        print("\n📊 GPX Statistics:")
        print(f"   Tracks: {len(tracks)}")
        print(f"   Segments: {len(segments)}")
        print(f"   Points: {len(points):,}")
        if tracks:
            print(f"   Avg points/track: {len(points) / len(tracks):.0f}")

        # Show first track info
        if tracks:
            first_track = tracks[0]
            name = first_track.find("gpx:name", ns)
            desc = first_track.find("gpx:desc", ns)
            print("\n   First track:")
            print(f"     Name: {name.text if name is not None else 'No name'}")
            if desc is not None and desc.text:
                desc_preview = desc.text[:100] + "..." if len(desc.text) > 100 else desc.text
                print(f"     Description: {desc_preview}")

        # Visualize GPX with individual track layers
        print("\n🗺️ Creating interactive map with track selection...")
        print("   ➡️ Click the layer control icon to show/hide tracks")

        # Create map
        if selected_bounds:
            center_lat = (selected_bounds[1] + selected_bounds[3]) / 2
            center_lon = (selected_bounds[0] + selected_bounds[2]) / 2
        else:
            # Use first point as center
            first_point = points[0] if points else None
            if first_point:
                center_lat = float(first_point.get("lat"))
                center_lon = float(first_point.get("lon"))
            else:
                center_lat, center_lon = 59.0, 10.0

        # Create map with topographic basemap
        gpx_map = folium.Map(
            location=[center_lat, center_lon],
            zoom_start=10,
            tiles=None,  # We'll add custom tiles
            prefer_canvas=True,
        )

        # Add topographic tile layer
        folium.TileLayer(
            tiles="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
            attr=(
                'Map data: &copy; <a href="https://openstreetmap.org/copyright">OpenStreetMap</a> contributors, '
                '<a href="http://viewfinderpanoramas.org">SRTM</a> | Map style: &copy; '
                '<a href="https://opentopomap.org">OpenTopoMap</a> '
                '(<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)'
            ),
            name="Topographic",
            overlay=False,
            control=True,
            max_zoom=17,
        ).add_to(gpx_map)

        # Add selected bounds
        if selected_bounds:
            bounds_rect = folium.Rectangle(
                bounds=[
                    [selected_bounds[1], selected_bounds[0]],
                    [selected_bounds[3], selected_bounds[2]],
                ],
                color="#0066FF",
                weight=3,
                fill=True,
                fillColor="#0066FF",
                fillOpacity=0.1,
                popup="Selected export area",
                name="📍 Export Area",
            )
            bounds_rect.add_to(gpx_map)

        # High contrast colors for topographic maps
        colors = [
            "#FF0000",  # Bright red
            "#00FF00",  # Bright green
            "#FF00FF",  # Magenta
            "#00FFFF",  # Cyan
            "#FFFF00",  # Yellow
            "#FF8800",  # Orange
            "#0088FF",  # Sky blue
            "#FF0088",  # Hot pink
            "#88FF00",  # Lime
            "#8800FF",  # Purple
            "#FF4444",  # Light red
            "#44FF44",  # Light green
            "#4444FF",  # Light blue
            "#FFAA00",  # Gold
            "#FF00AA",  # Deep pink
            "#00AAFF",  # Light cyan
            "#AA00FF",  # Violet
            "#00FFAA",  # Aquamarine
            "#AAFF00",  # Yellow-green
            "#FF5500",  # Dark orange
        ]

        # Add each track as a separate feature group
        print(f"\n   Adding {len(tracks)} tracks to map:")
        for i, track in enumerate(tracks):
            track_name = track.find("gpx:name", ns)
            track_name_text = track_name.text if track_name is not None else f"Track {i + 1}"

            # Extract trail number from track description if available
            track_desc = track.find("gpx:desc", ns)
            trail_number = None
            if track_desc is not None and track_desc.text:
                # Look for trail_number in description
                for line in track_desc.text.split("\n"):
                    if "trail_number:" in line:
                        trail_number = line.split("trail_number:")[1].strip()
                        break

            # Create display name with trail number if available
            if trail_number:
                display_name = f"{trail_number}: {track_name_text[:25]}"
            else:
                display_name = f"{i + 1}. {track_name_text[:30]}"

            if len(track_name_text) > 30:
                display_name += "..."

            # Create a feature group for this track - ALL shown by default
            track_group = folium.FeatureGroup(
                name=display_name,
                show=True,  # ALL tracks shown by default
                overlay=True,
                control=True,
            )

            # Get color for this track
            track_color = colors[i % len(colors)]

            # Get segments for this track
            track_segments = track.xpath(".//gpx:trkseg", namespaces=ns)

            for segment in track_segments:
                track_points = segment.xpath(".//gpx:trkpt", namespaces=ns)

                if track_points:
                    coords = []
                    for point in track_points:
                        lat = float(point.get("lat"))
                        lon = float(point.get("lon"))
                        coords.append([lat, lon])

                    # Add white background line for contrast
                    folium.PolyLine(
                        locations=coords,
                        color="white",
                        weight=5,
                        opacity=0.8,
                    ).add_to(track_group)

                    # Add colored trail line
                    folium.PolyLine(
                        locations=coords,
                        color=track_color,
                        weight=3,
                        opacity=1.0,
                        popup=folium.Popup(track_name_text, max_width=200),
                    ).add_to(track_group)

            track_group.add_to(gpx_map)

            if i < 5:
                print(f"     ✓ {display_name} - {track_color}")

        if len(tracks) > 5:
            print(f"     ... and {len(tracks) - 5} more tracks")

        print(f"   ✅ All {len(tracks)} tracks added and selected")

        # Add layer control - collapsed by default
        folium.LayerControl(
            collapsed=True,  # Start collapsed
            autoZIndex=True,
            position="topright",
        ).add_to(gpx_map)

        # Display map
        print("\n🗺️ Interactive Map (Topographic):")
        display(gpx_map)

    else:
        print("❌ GPX file not found")

print("\n" + "=" * 50)
print("✅ Verification complete!")